In [2]:
import os
import sys
import pandas as pd
import numpy as np
import time

In [3]:
HOME_DIR = os.path.expanduser('~/')
sys.path.append(f"{HOME_DIR}/Documents/Algo/Stock Price DB/")
from StockPriceData import process_ticker
script_path = HOME_DIR+'/Documents/Algo/Strategies/Using Pandas/Momentum/Multiple Timeframes'
cached_data = script_path+'/'+'cached_data'
stats_data = script_path+'/'+'stats'
data_folder = f'{HOME_DIR}/Documents/Algo/Data'
ticker_folder = f'{data_folder}/Tickers/Nasdaq-100'
current_date = time.strftime("%Y.%m.%d.%H%M%S")
daily_price_csv = f'{cached_data}/nasdaq100_daily_prices.csv'

In [4]:
all_daily_prices_df = pd.read_csv(daily_price_csv, index_col=['Date'], parse_dates=True)
print(f'Loaded prices for {all_daily_prices_df.shape[1]} tickers.')
daily_prices_df = all_daily_prices_df.copy()
daily_prices_df['CASH'] = 0.0 # We set to 0 to simulate 0% return as alternative to other investments when they're negative

# OLD METHODS
daily_returns = (daily_prices_df.pct_change()+1)[1:] # Adding 1 allows us to take the product of returns. Can't use cumsum though. This fucks things up for stocks that go to zero!
weekly_returns = daily_returns.resample('W').prod() # Need to subtract 1 to get actual % returns.
monthly_returns = daily_returns.resample('M').prod() # Need to subtract 1 to get actual % returns.

# NEW METHODS
daily_prices_df = daily_prices_df.loc[:,['ATVI','ADBE','ADP','ABNB','CASH']].copy()
daily_returns2 = (daily_prices_df.pct_change())[1:]
last_day_of_month = daily_prices_df.resample('M').agg(lambda x: x[-1]) # Resample to longer time frame, taking last value from each period using aggregation function
monthly_return_method1 = ((last_day_of_month / last_day_of_month.shift(1)) - 1).fillna(0) # Manual percentage change calculation
monthly_return_method2 = (last_day_of_month.pct_change()).fillna(0) # Using pandas method


Loaded prices for 206 tickers.


In [5]:
print(f'monthly_return_method1:\n {monthly_return_method1.iloc[:5,:5]}')
print(f'monthly_return_method2:\n {monthly_return_method2.iloc[:5,:5]}')
#print(f'daily_returns:\n  {daily_returns.iloc[:5,:5]}')
#print(f'daily_returns2:\n  {daily_returns2.iloc[:5,:5]}')
#print(f'weekly_returns:\n  {weekly_returns.iloc[:5,:5]}')
#print(f'weekly_returns2:\n  {weekly_returns2.iloc[:5,:5]}')

monthly_return_method1:
                 ATVI      ADBE       ADP  ABNB  CASH
Date                                                
2010-01-31  0.000000  0.000000  0.000000   0.0   0.0
2010-02-28  0.059933  0.072755  0.020244   0.0   0.0
2010-03-31  0.134031  0.020779  0.077125   0.0   0.0
2010-04-30 -0.080332 -0.050042 -0.024678   0.0   0.0
2010-05-31 -0.030120 -0.045238 -0.057377   0.0   0.0
monthly_return_method2:
                 ATVI      ADBE       ADP  ABNB  CASH
Date                                                
2010-01-31  0.000000  0.000000  0.000000   0.0   0.0
2010-02-28  0.059933  0.072755  0.020244   0.0   0.0
2010-03-31  0.134031  0.020779  0.077125   0.0   0.0
2010-04-30 -0.080332 -0.050042 -0.024678   0.0   0.0
2010-05-31 -0.030120 -0.045238 -0.057377   0.0   0.0


In [6]:
#all_daily_prices_df.head()
abnb_returns = monthly_return_method2['ABNB']
for date in abnb_returns.index:
    #print(f'{date}:  {abnb_returns[date]}')
    if abnb_returns[date] > 0 and abnb_returns[date-1*date.freq] == 0:
        print(f'Turned positive on {date}')

Turned positive on 2021-01-31 00:00:00


/tmp/ipykernel_60996/826244296.py:5: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if abnb_returns[date] > 0 and abnb_returns[date-1*date.freq] == 0:


In [7]:
abnb_returns['2020-11-30':'2021-03-31']

Date
2020-11-30    0.000000
2020-12-31    0.000000
2021-01-31    0.250886
2021-02-28    0.123727
2021-03-31   -0.089217
Freq: M, Name: ABNB, dtype: float64

In [19]:
parameter_combos = [{'stop_type': 'indiv', 'qty_long_period': 30, 'qty_med_period': 20, 'qty_short_period': 5, 'SMA_S': 1, 'SMA_L': 146, 'rolling_l': 11, 'rolling_m': 9, 'rolling_s': 4, 'stop': 41},
                    {'stop_type': 'indiv', 'qty_long_period': 30, 'qty_med_period': 20, 'qty_short_period': 5, 'SMA_S': 10, 'SMA_L': 15, 'rolling_l': 11, 'rolling_m': 4, 'rolling_s': 9, 'stop': 41},
                    {'stop_type': 'indiv', 'qty_long_period': 30, 'qty_med_period': 20, 'qty_short_period': 5, 'SMA_S': 10, 'SMA_L': 1, 'rolling_l': 11, 'rolling_m': 6, 'rolling_s': 3, 'stop': 41},
                    ]

In [20]:
parameter_combos

[{'stop_type': 'indiv',
  'qty_long_period': 30,
  'qty_med_period': 20,
  'qty_short_period': 5,
  'SMA_S': 1,
  'SMA_L': 146,
  'rolling_l': 11,
  'rolling_m': 9,
  'rolling_s': 4,
  'stop': 41},
 {'stop_type': 'indiv',
  'qty_long_period': 30,
  'qty_med_period': 20,
  'qty_short_period': 5,
  'SMA_S': 10,
  'SMA_L': 15,
  'rolling_l': 11,
  'rolling_m': 4,
  'rolling_s': 9,
  'stop': 41},
 {'stop_type': 'indiv',
  'qty_long_period': 30,
  'qty_med_period': 20,
  'qty_short_period': 5,
  'SMA_S': 10,
  'SMA_L': 1,
  'rolling_l': 11,
  'rolling_m': 6,
  'rolling_s': 3,
  'stop': 41}]

In [21]:
parameter_combos_df = (pd.DataFrame(parameter_combos, columns=parameter_combos[0].keys())) # Create dataframe from list of dictionaries
print(f'\nparameter_combos_df:\n{parameter_combos_df}')



parameter_combos_df:
  stop_type  qty_long_period  qty_med_period  qty_short_period  SMA_S  SMA_L  \
0     indiv               30              20                 5      1    146   
1     indiv               30              20                 5     10     15   
2     indiv               30              20                 5     10      1   

   rolling_l  rolling_m  rolling_s  stop  
0         11          9          4    41  
1         11          4          9    41  
2         11          6          3    41  


In [23]:

# Iterate all rows using DataFrame.iterrows()
for index, parameter_combo in parameter_combos_df.iterrows():
    print (parameter_combo['rolling_l'], parameter_combo['rolling_m'], parameter_combo['rolling_s'])


11 9 4
11 4 9
11 6 3


In [28]:
parameter_combos_df = parameter_combos_df.loc[(parameter_combos_df['rolling_l'] >= parameter_combos_df['rolling_m']) & (parameter_combos_df['rolling_m'] >= parameter_combos_df['rolling_s']) & (parameter_combos_df['SMA_L'] >= parameter_combos_df['SMA_S'])]
print(f'\nparameter_combos_df:\n{parameter_combos_df}')


parameter_combos_df:
  stop_type  qty_long_period  qty_med_period  qty_short_period  SMA_S  SMA_L  \
0     indiv               30              20                 5      1    146   

   rolling_l  rolling_m  rolling_s  stop  
0         11          9          4    41  
